In [1]:
import cv2
import numpy as np
import webbrowser
import torch
import os
import pandas as pd
import pickle

# I. Définition des fonctions 

In [4]:
# vidéo callback

def onMouse(event, x, y, flags, links):
    """
    For each frame : 
    When mouse inside, display clickable area in image
    Open link in web browser when image clicked
    
    input : 
        x,y : position of the mouse on the image 
        links : Links object. Allows  to make the image interactive

    """
    if event == cv2.EVENT_LBUTTONDOWN:
        #check if item has been clicked
        links.itemClicked(x,y) 
        
    if event == cv2.EVENT_MOUSEMOVE: 
        # check if mouse inside a clickable area
        links.displayArea(x,y)

In [5]:


def itemsExtraction(model, path='advertising.mp4'):
    """
    for each frame, model detect clothes and return their positions
    """

    adVideo = cv2.VideoCapture(path)
    #check if vidéo 
    if (adVideo.isOpened()== False): 
      print("Error opening video stream or file")


    frame_index = 0
    data = { }
    # Read until video is completed
    while(adVideo.isOpened()):
      # Capture frame-by-frame
      ret, frame = adVideo.read()
      if ret == True:

        #detect clothes
        results = model(frame)   
        if results.pandas().xyxy[0].shape[0] == 0 :
            #if no clothes detetected return empty value 
            d = {'xmin': [0], 'ymin': [0], 'xmax': [0], 'ymax': [0], 'confidence': [0], 'class': [0], 'name': [" "]}
            data[frame_index] = np.array(pd.DataFrame(data=d))[0]
        else : 
            #return model result => for each frame only detect one clothes
            data[frame_index] = np.array(results.pandas().xyxy[0].iloc[results.pandas().xyxy[0]['confidence'].argmax()])
        frame_index += 1



      else: 
        break

    # When everything done, release the video capture object

    adVideo.release()
    if (adVideo.isOpened()== False): 
      print("Vidéo closed")

    # Closes all the frames
    cv2.destroyAllWindows()


    return data

In [8]:
class Links:
    """
    Class that makes video clickable
    """
        
    def __init__(self, data, items):
        """
        data : clothes bbox for each frame. Result of itemsExtraction function
        items : links associated with clothes
        frame_index : index of the actual frame
        display :bool to display area'border only when mouse inside
        """
        self.data = data
        self.items = items
        self.frame_index = 0
        self.display = False
            
            

               
    def itemClicked(self,x,y): 
        
        """
        method who detect which clothes has been clicked, and open the corresponding link
        """
        
        #check if mouse inside the area : if inside open the link
        if( (x > self.data[self.frame_index][0]) and (x < self.data[self.frame_index][2])
           and (y > self.data[self.frame_index][1]) and (y < self.data[self.frame_index][3])):
#                 webbrowser.open(items[self.data[frame][6]])
              webbrowser.open('https://fr.shein.com/Men-Solid-Notched-Neck-Tee-p-9079332-cat-1978.html?url_from=fradplasm2112064364522856M_GPM&cid=16923873116&setid=&adid=&pf=GOOGLE&gclid=CjwKCAjwwo-WBhAMEiwAV4dybdFPPKeAXk-t6suc5z_-2aqXyCbqWfYIHA3xfRMTYOlTExBxcQFPSxoC8IwQAvD_BwE')
                
                
    def displayArea(self, x, y):
        """
        method who detect if mouse in clickable area. If inside, display border of this area as insight for the customer he 
        can click
        """
        

            
        if((x > self.data[self.frame_index][0]) and (x < self.data[self.frame_index][2])
           and (y > self.data[self.frame_index][1]) and (y < self.data[self.frame_index][3])):

            if(not self.display):    
                # if mouse inside area, display the area
                self.display = True

        else : 
            if (self.display):
                # if mouse quit area stop displaying the area
                self.display = False
        
        
    def incrementFrameIndex(self): 
        
        self.frame_index += 1
        
    
    def getItems(self):
        #get position of clothes for the current frame
        
        return (self.data[self.frame_index])
       
                
    def clearLinks(self): 
        #clear the object
        self.data = {}
        self.items = {}
        self.frame_index = 0
        self.display = False
        
    def reset(self): 
        #reset the object to be use again
        self.frame_index = 0
        self.display = False

In [9]:


def data_process(dataset, distance=5, windowAvgSize=5):
    """
    Process the data. Fills frames that have no items when they should have them.
    Average of the bboxes among the frames to avoid a too important movement of the bboxes.
    
    dataset : bbox clothing for each image. Result of the itemsExtraction function
    distance : maximum abnormal distance of frames without elements. If the distance is smaller, we fill the blanks. 
    If the distance is greater we consider it to be a frames sequence without clothes.
    windowAVgSize : Size of the window to average the bboxes among
    
    
    """
    
    data = dataset.copy()
    
    
    #check for undetected clothes. 
    for index in range(1,len(data)-1) : 
        
        #detect frame without items
        if np.sum(data[index][:4] == 0):
            space_index = 1
            
            #check the length of the frame sequence without items
            while np.sum(data[space_index + index][:4] == 0) and space_index + index < len(data) - 2:
                space_index += 1
                               
            #if sequence of frames whitout items short enough and items detected before and after are the same, fill 
            #the blanks
            if space_index <= distance and np.mean(np.std(np.append(data[index - 1][:4][None, :]
             ,data[index + space_index][:4][None, :], axis=0).astype('float64'), axis = 0)) < 10:

                for k in range(space_index):
                    data[index + k] = data[index - 1]
                    
    
    #average on sequence of frames of similar items
    for index in range(0, len(data) - windowAvgSize) : 
        
        dataNeibhor = data[index][:4][None, :]
        
        for k in range(1, windowAvgSize) : 
            
            dataNeibhor = np.append(dataNeibhor, data[index + k][:4][None, :], axis=0)
        
        #check if similar items
        if np.mean(np.std(dataNeibhor.astype('float64'), axis = 0)) < 10:

            data[index][0] = np.mean(dataNeibhor, axis=0)[0]
            data[index][1] = np.mean(dataNeibhor, axis=0)[1]
            data[index][2] = np.mean(dataNeibhor, axis=0)[2]
            data[index][3] = np.mean(dataNeibhor, axis=0)[3]
        
            
        

                
          
    return data

# II. Proto

In [16]:
#import model 

model = torch.hub.load('../iaApi/yolov5', 'custom', path='../iaApi/models/Fashion_Datasets_50_labels_V1.pt', source='local')
model.conf = 0.1


YOLOv5  2022-5-20 Python-3.9.12 torch-1.11.0+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7144975 parameters, 0 gradients, 16.2 GFLOPs
Adding AutoShape... 


In [17]:
#data and data manipulation

# data = itemsExtraction(model, path='advertising.mp4')
# data = data_process(data)

# fileObj = open('data.obj', 'wb')
# pickle.dump(data,fileObj)
# fileObj.close()


fileObj = open('data.obj', 'rb')
data = pickle.load(fileObj)
fileObj.close()

In [18]:
links = Links(data, {})

In [19]:
#main : application demo


adVideo = cv2.VideoCapture('advertising.mp4')
if (adVideo.isOpened()== False): 
  print("Error opening video stream or file")


color = (255, 0, 0)
thickness = 2
links = Links(data, {})


# Read until video is completed
while(adVideo.isOpened()):
  # Capture frame-by-frame
  ret, frame = adVideo.read()
  if ret == True:
    
    # Window name in which image is displayed
    window_name = 'Image'

   
    

   
    #show clickable are
    if(links.display):
        itemsData = links.getItems()
        frame = cv2.rectangle(frame, (int(itemsData[0]),int(itemsData[1])),
                              (int(itemsData[2]),int(itemsData[3])), color,thickness)

    

    # Display the resulting frame
    cv2.imshow('Frame',frame)   
    cv2.setMouseCallback('Frame', onMouse, links)
    
    links.incrementFrameIndex()

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
    


  # Break the loop
  else: 
    break

# When everything done, release the video capture object
adVideo.release()
if (adVideo.isOpened()== False): 
  print("Vidéo closed")

# Closes all the frames
cv2.destroyAllWindows()
links.reset()



Vidéo closed
